In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [5]:
# filter df for total_votes > 20
df_2 = df.filter(df.total_votes >= 20)
df_2.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17728960|R14W2VCHHK5V7W|B00ZI3R5VG|     213303319|Stuhrling Origina...|         Watches|          5|           19|         20|   N|                Y|this is a good al...|As a former Rolex...| 2015-08-31|
|         US|   20240757|R1S3T57O3OYT5S|B003DZ7VOW|     336197576|G-Shock GA100A-7A...|         Watches|          5|    

In [6]:
# Filter df_2 for helpful_votes > 50%
df_3 = df_2.filter((df_2.helpful_votes / df_2.total_votes) >= 0.5)
df_3.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17728960|R14W2VCHHK5V7W|B00ZI3R5VG|     213303319|Stuhrling Origina...|         Watches|          5|           19|         20|   N|                Y|this is a good al...|As a former Rolex...| 2015-08-31|
|         US|   20240757|R1S3T57O3OYT5S|B003DZ7VOW|     336197576|G-Shock GA100A-7A...|         Watches|          5|    

In [7]:
#create paid vine df
paid_vine_df = df_3.filter(df_3.vine == 'Y')
paid_vine_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53096363|R1B7M0OP3UNP6O|B00V0G9OGE|     297682929|Armitron Sport Wo...|         Watches|          5|           49|         52|   Y|                N|Lots of features ...|My wife has been ...| 2015-08-27|
|         US|   51394788|R2UUV4UGGYMQG8|B00M1Y5AIU|     368146384|Boulevard ClickTi...|         Watches|          5|    

In [8]:
#create unpaid vine df
unpaid_vine_df = df_3.filter(df_3.vine == "N")
unpaid_vine_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17728960|R14W2VCHHK5V7W|B00ZI3R5VG|     213303319|Stuhrling Origina...|         Watches|          5|           19|         20|   N|                Y|this is a good al...|As a former Rolex...| 2015-08-31|
|         US|   20240757|R1S3T57O3OYT5S|B003DZ7VOW|     336197576|G-Shock GA100A-7A...|         Watches|          5|    

In [11]:
#paid total number of reviews
total_paid_review = paid_vine_df.count()
total_paid_review

47

In [12]:
#unpaid total number of reviews
total_unpaid_review = unpaid_vine_df.count()
total_unpaid_review

8362

In [13]:
# paid 5 star review
paid_5_star = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
paid_5_star

15

In [14]:
# unpaid 5 star review
unpaid_5_star = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
unpaid_5_star

4332

In [15]:
# paid 5 star review ratio
paid_5_star_ratio = (paid_5_star / total_paid_review) *100
paid_5_star_ratio

31.914893617021278

In [16]:
# unpaid 5 star review ratio
unpaid_5_star_ratio = (unpaid_5_star / total_unpaid_review) *100
unpaid_5_star_ratio

51.80578808897393